In [ ]:
!pip install pygooglenews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 31.7 MB/s 
     |████████████████████████████████| 1.2 MB 41.8 MB/s 
     |████████████████████████████████| 128 kB 43.8 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44952 sha256=9bf2ed5641befe643d3e9392b5467a432252e3c2d1f241f0ee87aaf45d5bd06f
  Stored in directory: /root/.cache/pip/wheels/29/bf/46/b4a597d435d3aee6c2fa583824897336d65abf13ebe3405b70
Successfully built feedparser
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from pygooglenews import GoogleNews
from bs4 import BeautifulSoup

import requests
import os
import re
import math
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/IT458_project/"

Mounted at /content/gdrive


# Data Extraction

In [ ]:
gn = GoogleNews(lang='en', country='IN')
#list_of_search_terms = ["gun violence","gun death","gun control","gun fire","gun killed","gun injured","gun shot"]
s = gn.search('india gun killed')

In [ ]:
news = []

for i in range(0,len(s['entries'])):

  # Get title of recent gun violence news
  title = s['entries'][i].title
  t = len(title)-title[::-1].index('-')-1
  article_name = title[t+2:]

  # Get url of recent gun violence news
  url = s['entries'][i].links[0].href
  
  # Get the published date recent gun violence news in format MM-DD-YYYY
  date = s['entries'][i].published
  d = date.split(" ")
  n_month = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
  preprocessed_date = n_month[d[2]] + '-' + d[1] + '-' + d[3]

  if(url[0:4]=="http"):
    news.append([article_name,title[0:t],url,preprocessed_date])

news = sorted(news,key=lambda x:x[3])[::-1]

In [ ]:
for i in news:
  print(i)

['Al Jazeera English', 'Suspected rebel killed in fresh Kashmir gun battle ', 'https://news.google.com/__i/rss/rd/articles/CBMiUmh0dHBzOi8vd3d3LmFsamF6ZWVyYS5jb20vbmV3cy8yMDIxLzEyLzE0L2luZGlhLWthc2htaXItc3VzcGVjdGVkLXJlYmVsLWd1bi1iYXR0bGXSAQA?oc=5', '12-14-2021']
['Al Jazeera English', 'Dozens of Maoist rebels killed in India gun battle ', 'https://news.google.com/__i/rss/rd/articles/CBMiXGh0dHBzOi8vd3d3LmFsamF6ZWVyYS5jb20vbmV3cy8yMDIxLzExLzEzL2F0LWxlYXN0LTI2LW1hb2lzdC1yZWJlbHMta2lsbGVkLWluLWluZGlhLWd1bmZpZ2h00gEA?oc=5', '11-13-2021']
['Times Now', "Sidhu Moose Wala's song 'Vaar' releases on YouTube, sends shockwaves of tributes and emotions across platforms ", 'https://news.google.com/__i/rss/rd/articles/CBMingFodHRwczovL3d3dy50aW1lc25vd25ld3MuY29tL3ZpcmFsL3NpZGh1LW1vb3NlLXdhbGFzLXNvbmctdmFhci1yZWxlYXNlcy1vbi15b3V0dWJlLXNlbmRzLXNob2Nrd2F2ZXMtb2YtdHJpYnV0ZXMtYW5kLWVtb3Rpb25zLWFjcm9zcy1wbGF0Zm9ybXMtYXJ0aWNsZS05NTM3NjU2MtIBAA?oc=5', '11-08-2022']
['Al Jazeera English', 'Myanmar’s anti-co

In [ ]:
import csv
 
header = ["Name of the article","Title of the news","URL link","Published date"]
 
# opening the csv file in 'a+' mode
file = open('news.csv', 'a+', newline ='')
 
# writing the data into the file
with file:
    writer = csv.DictWriter(file, fieldnames = header)
     
    # writing data row-wise into the csv file
    writer.writeheader()
    write.writerows(data)

In [ ]:
output_dir = 'output'

try:
  os.mkdir(path+output_dir)
  os.mkdir(path+output_dir+'/html_files')
  os.mkdir(path+output_dir+'/text_files')
except:
  print(end="")

In [ ]:
for i in range(len(news)):
  url = news[i][2]
  r = requests.get(url, allow_redirects=True)
  open(path + output_dir + '/html_files/article' + str(i+1) + '.html', 'wb').write(r.content)

print("Downloaded all news artile in html format")

Downloaded all news artile in html format


In [ ]:
files = os.listdir(path+output_dir+'/html_files')
texts = []

for eachFile in files:
  file = path+output_dir+'/html_files/' + eachFile
  if(os.path.isfile(file)):
    fileName = open(file, 'r+', encoding='utf-8')
    data = fileName.read()

    soup = BeautifulSoup(data,'html.parser')

    # to remove formualae present in each document
    for formulae in soup('semantics'):
      formulae.decompose()
    text_data = ""

    #consider texts between only p,h1,h2,h3,h4 tags
    for data in soup.find_all(['p','h1','h2','h3','h4']):
      text_data = text_data + data.text + " "

    #to remove utf-8 data
    text_data= str(text_data.encode('ascii','ignore').decode('ascii'))
    text_data = text_data.split()
    formatted_data = " ".join(text_data)

    # regex to remove extra spaces between the words
    formatted_data = re.sub('\s\s+', ' ', formatted_data)
    formatted_data = re.sub("[\"#!$%&()*+/:;<=>?@[\]_`{|}^~?-]","",formatted_data)
    formatted_data = re.sub('(?<=[^0-9])[.,\']|[.,\'](?=[^0-9])', '',formatted_data)
    formatted_data = formatted_data.replace(",","")
    formatted_data = formatted_data .replace(".","")
    formatted_data = formatted_data.lower()
    texts.append(formatted_data)

    newFileName = eachFile.split(".html")
    f = open(path + output_dir + '/text_files/' + newFileName[0] + '.txt', 'w+')
    f.write(formatted_data)
    f.close()

print("Text document generated for all news article")

Text document generated for all news article


# Evaluation with each model

In [ ]:
def convert_examples_to_features(text, tokenizer, max_seq_length):
    token_batch = []
    tokens = tokenizer.tokenize(text, truncation=True, max_length=max_seq_length)

In [ ]:
def evaluate_crf(model, evaluate_X, evaluate_Y, tokenizer, cuda_available, batch_size, max_seq_length, model_type, lr, epochs, path, get_accuracy):

    model.eval()
    num_samples = len(evaluate_X)
    evaluate_set = GunViolenceDataset(evaluate_X, evaluate_Y)
    evaluate_generator = DataLoader(
        evaluate_set,
        batch_size=1,
        shuffle=True,
    )
    num_of_tp = num_of_fn = num_of_fp = num_of_tn = 0

    for i, (evaluate_x, evaluate_y) in enumerate(evaluate_generator):
        tokens, labels = convert_examples_to_features(evaluate_x, evaluate_y, tokenizer, max_seq_length)

        indexed_tokens = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
        segments_ids = [[0] * len(indexed_token) for indexed_token in indexed_tokens]

        if cuda_available:
            segments_tensors = torch.tensor(segments_ids).to('cuda')
            tokens_tensor = torch.tensor(indexed_tokens).to('cuda')
            labels = torch.tensor(labels).to('cuda')
        else:
            segments_tensors = torch.tensor(segments_ids)
            tokens_tensor = torch.tensor(indexed_tokens)
            labels = torch.tensor(labels)

        with torch.no_grad():
            y_pred, logits, loss = model(tokens_tensor, segments_tensors, labels)
            normalized_probs = nn.functional.softmax(logits, dim=1)[0]
            results = y_pred[0]

            # get the real target
            original = ''
            for i, (x, y) in enumerate(zip(evaluate_x[0].split(), evaluate_y[0].split())):
                if y[0] == 'B':
                    original = x + ' '
                    index = i
                    while index + 1 < len(evaluate_y[0].split()) and evaluate_y[0].split()[index + 1][0] == 'I':
                        original += '{} '.format(evaluate_x[0].split()[index + 1])
                        index += 1
                    break
            original = original.strip()

            probabilities = []
            predictions = []
            prediction = []

            for token, tag, prob in zip(tokens[0], results, normalized_probs):
                if tag == 0:
                    # tag == 'B'
                    probabilities.append(prob)

                    if len(prediction) != 0:
                        predictions.append(prediction)
                        prediction = []
                    prediction.append(token)
                elif tag == 1:
                    # tag == 'I'
                    prediction.append(token)
            if len(prediction) != 0:
                predictions.append(prediction)

            # one sentence might generate multiple targets, eg. shooters or victims
            # we need to pick the most possible one, which is the one has the highest probability in 'B' tag
            max_prob = -math.inf
            max_prob_ind = 0
            for i, prob in enumerate(probabilities):
                if max_prob < prob[0]:
                    max_prob_ind = i
                    max_prob = prob[0]

            if(len(predictions)!=0):
                result.append(tokenizer.convert_tokens_to_string(predictions[max_prob_ind]))
            else:
                result.append('')
    
    return result    

In [ ]:
def evaluate(model, evaluate_X, evaluate_Y, tokenizer, cuda_available, batch_size, max_seq_length, model_type, lr, epochs, path, get_accuracy):

    model.eval()
    num_samples = len(evaluate_X)
    evaluate_set = GunViolenceDataset(evaluate_X, evaluate_Y)
    evaluate_generator = DataLoader(
        evaluate_set,
        batch_size=1,
        shuffle=True,
    )
    result = []

    for i, (evaluate_x, evaluate_y) in enumerate(evaluate_generator):
        tokens, labels = convert_examples_to_features(evaluate_x, evaluate_y, tokenizer, max_seq_length)

        indexed_tokens = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
        segments_ids = [[0] * len(indexed_token) for indexed_token in indexed_tokens]

        if cuda_available:
            segments_tensors = torch.tensor(segments_ids).to('cuda')
            tokens_tensor = torch.tensor(indexed_tokens).to('cuda')
            labels = torch.tensor(labels).to('cuda')
        else:
            segments_tensors = torch.tensor(segments_ids)
            tokens_tensor = torch.tensor(indexed_tokens)
            labels = torch.tensor(labels)

        with torch.no_grad():
            y_pred, logits, loss = model(tokens_tensor, segments_tensors, labels)
            normalized_probs = nn.functional.softmax(logits, dim=1)[0]
            results = y_pred[0]

            # get the real target
            original = ''
            for i, (x, y) in enumerate(zip(evaluate_x[0].split(), evaluate_y[0].split())):
                if y[0] == 'B':
                    original = x + ' '
                    index = i
                    while index + 1 < len(evaluate_y[0].split()) and evaluate_y[0].split()[index + 1][0] == 'I':
                        original += '{} '.format(evaluate_x[0].split()[index + 1])
                        index += 1
                    break
            original = original.strip()

            probabilities = []
            predictions = []
            prediction = []

            for token, tag, prob in zip(tokens[0], results, normalized_probs):
                if tag == 0:
                    # tag == 'B'
                    probabilities.append(prob)

                    if len(prediction) != 0:
                        predictions.append(prediction)
                        prediction = []
                    prediction.append(token)
                elif tag == 1:
                    # tag == 'I'
                    prediction.append(token)
            if len(prediction) != 0:
                predictions.append(prediction)

            # one sentence might generate multiple targets, eg. shooters or victims
            # we need to pick the most possible one, which is the one has the highest probability in 'B' tag
            max_prob = -math.inf
            max_prob_ind = 0
            for i, prob in enumerate(probabilities):
                if max_prob < prob[0]:
                    max_prob_ind = i
                    max_prob = prob[0]
    
            if(len(predictions)!=0):
                result.append(tokenizer.convert_tokens_to_string(predictions[max_prob_ind]))
            else:
                result.append('')
    
    return result        

In [ ]:
TRANSFORMER_PATH = 'huggingface/pytorch-transformers'
tokenizer = torch.hub.load(TRANSFORMER_PATH, 'tokenizer', 'bert-base-cased')
cuda_available = torch.cuda.is_available()
batch_size = 40
max_seq_length = 256
lr = 1e-4
epochs = 7

BERT_Linear

In [ ]:
output_path = path + 'output_model/victim_linear'
model = torch.load(output_path)
print(evaluate(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'Linear', lr, epochs, output_path))

In [ ]:
output_path = path + 'output_model/shooter_linear'
model = torch.load(output_path)
print(evaluate(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'Linear', lr, epochs, output_path))

BERT_LSTM

In [ ]:
output_path = path + 'output_model/victim_lstm'
model = torch.load(output_path)
print(evaluate(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'LSTM', lr, epochs, output_path))

In [ ]:
output_path = path + 'output_model/shooter_lstm'
model = torch.load(output_path)
print(evaluate(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'LSTM', lr, epochs, output_path))

BERT_BiLSTM

In [ ]:
output_path = path + 'output_model/victim_bilstm'
model = torch.load(output_path)
print(evaluate(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'BiLSTM', lr, epochs, output_path))

In [ ]:
output_path = path + 'output_model/shooter_Bilstm'
model = torch.load(output_path)
print(evaluate(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'BiLSTM', lr, epochs, output_path))

BERT_CRF_Linear

In [ ]:
output_path = path + 'output_model/victim_linear_crf'
model = torch.load(output_path)
print(evaluate_crf(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'Linear', lr, epochs, output_path))

In [ ]:
output_path = path + 'output_model/shooter_linear_crf'
model = torch.load(output_path)
print(evaluate_crf(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'Linear', lr, epochs, output_path))

BERT_CRF_LSTM

In [ ]:
output_path = path + 'output_model/victim_lstm_crf'
model = torch.load(output_path)
print(evaluate_crf(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'LSTM', lr, epochs, output_path))

In [ ]:
output_path = path + 'output_model/shooter_lstm_crf'
model = torch.load(output_path)
print(evaluate_crf(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'LSTM', lr, epochs, output_path))

BERT_CRF_BiLSTM

In [ ]:
output_path = path + 'output_model/victim_bilstm_crf'
model = torch.load(output_path)
print(evaluate_crf(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'BiLSTM', lr, epochs, output_path))

In [ ]:
output_path = path + 'output_model/shooter_bilstm_crf'
model = torch.load(output_path)
print(evaluate_crf(model, texts, [], tokenizer, cuda_available, batch_size, max_seq_length,'BiLSTM', lr, epochs, output_path))